In [2]:
import os

In [3]:
%pwd

'c:\\Users\\Portable\\OneDrive\\Desktop\\Automated-Stock-Price-Prediction-Pipeline-A-Complete-MLOps-Lifecycle\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\Portable\\OneDrive\\Desktop\\Automated-Stock-Price-Prediction-Pipeline-A-Complete-MLOps-Lifecycle'

In [6]:
import joblib

In [7]:
from keras.models import load_model

model = load_model(r"artifacts\model\trained_model.h5")

In [8]:
model.summary()  

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 128)           66560     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 25)                1625      
                                                                 
 dense_1 (Dense)             (None, 1)                 26        
                                                                 
Total params: 117619 (459.45 KB)
Trainable params: 117619 (459.45 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [219]:
def _create_sequences(data: np.ndarray):
    x, y = [], []
    for i in range(60, len(data)):
        x.append(data[i - 60:i, 0])
        y.append(data[i, 0])
    x = np.array(x).reshape((-1, 60, 1))
    y = np.array(y)
    return x, y

In [220]:
scaled_data, dataset = evaluator.fetch_and_preprocess_real_time_data()

[2024-11-25 22:45:43,512: INFO: 1069016221: Real-time data fetched successfully.]
[2024-11-25 22:45:43,513: INFO: 1069016221: Real-time data fetched for symbol: IBM]
[2024-11-25 22:45:43,513: INFO: 1069016221: Real-time data scaled.]


In [221]:
scaled_data.shape

(100, 1)

In [222]:
x, y = _create_sequences(scaled_data)

In [223]:
y

array([0.47297297, 0.33783784, 0.47297297, 0.40540541, 0.40540541,
       0.47297297, 0.92567568, 0.65540541, 0.67567568, 0.49324324,
       0.5       , 0.47297297, 0.47972973, 0.31756757, 0.47297297,
       0.50675676, 0.47972973, 0.54054054, 0.31756757, 0.47972973,
       0.50777027, 0.54054054, 0.51351351, 0.50675676, 0.47972973,
       0.47972973, 0.47972973, 0.43918919, 0.54054054, 0.40540541,
       0.55398649, 0.54054054, 0.54054054, 0.54054054, 0.54054054,
       0.43918919, 0.54054054, 0.54054054, 0.54054054, 0.55405405])

In [224]:
x.shape

(40, 60, 1)

In [225]:
predictions = model.predict(x)

2/2 [==============================] - 1s 26ms/step


In [226]:
predictions

array([[0.34927928],
       [0.36449555],
       [0.37387547],
       [0.3841495 ],
       [0.39180982],
       [0.39732075],
       [0.4038615 ],
       [0.42974225],
       [0.45719242],
       [0.48548228],
       [0.505279  ],
       [0.5183438 ],
       [0.5248106 ],
       [0.52662927],
       [0.51843333],
       [0.51006734],
       [0.50338507],
       [0.49726802],
       [0.4944725 ],
       [0.48495683],
       [0.4778215 ],
       [0.47375503],
       [0.47348833],
       [0.47484496],
       [0.47700578],
       [0.47846118],
       [0.479297  ],
       [0.4796082 ],
       [0.47783253],
       [0.4788536 ],
       [0.4762055 ],
       [0.47727832],
       [0.48035207],
       [0.4847792 ],
       [0.489966  ],
       [0.4954372 ],
       [0.49662554],
       [0.4989123 ],
       [0.5018234 ],
       [0.5051058 ]], dtype=float32)

In [227]:
y

array([0.47297297, 0.33783784, 0.47297297, 0.40540541, 0.40540541,
       0.47297297, 0.92567568, 0.65540541, 0.67567568, 0.49324324,
       0.5       , 0.47297297, 0.47972973, 0.31756757, 0.47297297,
       0.50675676, 0.47972973, 0.54054054, 0.31756757, 0.47972973,
       0.50777027, 0.54054054, 0.51351351, 0.50675676, 0.47972973,
       0.47972973, 0.47972973, 0.43918919, 0.54054054, 0.40540541,
       0.55398649, 0.54054054, 0.54054054, 0.54054054, 0.54054054,
       0.43918919, 0.54054054, 0.54054054, 0.54054054, 0.55405405])

In [ ]:
def test_model(scaled_data: np.ndarray, training_data_len: int, dataset: np.ndarray):
    try:
        x_test, y_test = _create_sequences(scaled_data)
        print(f"x_test shape: {x_test.shape}")
        model = load_model()  
        predictions = model.predict(x_test)
        predictions = predictions.flatten()
        y_test = self.scaler.inverse_transform(y_test)
        rmse = np.sqrt(np.mean((predictions - y_test) ** 2))
        
        logger.info(f"Testing complete. RMSE: {rmse}")
        return rmse, predictions
    except Exception as e:
        logger.error(f"Error during model testing: {e}")
        raise

In [5]:
from dataclasses import dataclass
from pathlib import Path

    
    
@dataclass(frozen=True)
class ModelEvaluationConfig:
    model_path: Path
    sequence_length: int
    mlflow_uri: str
    symbol: str
    interval: str
    outputsize: str
    base_url: str
    api_key: str


In [6]:
from src.StockPricePrediction.utils.common import read_yaml, create_directories
from src.StockPricePrediction.constants import *

from dotenv import load_dotenv
from pathlib import Path
import os

load_dotenv()  # Load environment variables


class ConfigurationManager:
    def __init__(self, config_filepath: Path):
        self.config = read_yaml(config_filepath)

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config['model_evaluation_params']
        
        model_evaluation_config = ModelEvaluationConfig(
            model_path=Path(config['model_path']),
            sequence_length=config['sequence_length'],
            mlflow_uri=config['mlflow_uri'],
            symbol=config['symbol'],
            interval=config['interval'],
            outputsize=config['outputsize'],
            base_url=config['api_endpoint'],
            api_key=os.getenv('ALPHA_VANTAGE_API_KEY'),

        )
        
        return model_evaluation_config

In [ ]:
import os
import numpy as np
import pandas as pd
import requests  # Added missing import
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler  # Added missing import
from sklearn.metrics import mean_squared_error
import mlflow
import mlflow.keras
from urllib.parse import urlparse

from src.StockPricePrediction import logger


class ModelEvaluator:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        self.scaler = MinMaxScaler(feature_range=(0, 1))  # Initialized scaler

    def fetch_real_time_data(self) -> pd.DataFrame:
        try:
            params = {
                "function": "TIME_SERIES_INTRADAY",
                "symbol": self.config.symbol,
                "interval": self.config.interval,
                "apikey": self.config.api_key,
                "datatype": "json"
            }
            response = requests.get(self.config.base_url, params=params)
            response.raise_for_status()

            data = response.json()
            key = f"Time Series ({self.config.interval})"
            if key not in data:
                raise ValueError(f"Unexpected response format: {data}")

            # Extract and format data
            time_series = data[key]
            df = pd.DataFrame.from_dict(time_series, orient="index")
            df.columns = ["open", "high", "low", "close", "volume"]
            df = df.astype(float)
            df.index = pd.to_datetime(df.index)
            df.sort_index(inplace=True)
            logger.info("Real-time data fetched successfully.")
            return df
        except Exception as e:
            logger.error(f"Error fetching real-time data: {e}")
            raise

    def fetch_and_preprocess_real_time_data(self) -> np.ndarray:
        try:
            # Fetch real-time data
            real_time_data = self.fetch_real_time_data()
            logger.info(f"Real-time data fetched for symbol: {self.config.symbol}")

            # Use the 'close' column for predictions
            dataset = real_time_data[['close']].values

            # Scale data
            scaled_data = self.scaler.fit_transform(dataset)  # Use saved scaler if available
            logger.info("Real-time data scaled.")
            return scaled_data, dataset
        except Exception as e:
            logger.error(f"Error fetching and preprocessing real-time data: {e}")
            raise

    def test_model_real_time(self, scaled_data: np.ndarray):
        try:
            # Create sequences
            x_real_time, _ = self._create_sequences(scaled_data)

            # Predict
            model = load_model(self.config.model_path)  # Load model dynamically
            predictions = model.predict(x_real_time)
            
            predictions = self.scaler.inverse_transform(predictions)

            logger.info("Real-time model testing complete.")
            return predictions
        except Exception as e:
            logger.error(f"Error during real-time model testing: {e}")
            raise
    
    def _create_sequences(self, data: np.ndarray):
        x, y = [], []
        for i in range(60, len(data)):
            x.append(data[i - 60:i, 0])
            y.append(data[i, 0])
        x = np.array(x).reshape((-1, 60, 1))
        y = np.array(y)
        return x, y

    def test_model(self, scaled_data: np.ndarray, training_data_len: int, dataset: np.ndarray):
        try:
            x_test, y_test = self._create_sequences(scaled_data)
            print(f"x_test shape: {x_test.shape}")

            model = load_model(self.config.model_path)  

            predictions = model.predict(x_test)
            predictions = self.scaler.inverse_transform(predictions)
            predictions = predictions.flatten()
            y_test = self.scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
            print(f"prediction mean: {predictions.mean()}")
            print(f"y_test mean: {y_test.mean()}")
            rmse = np.sqrt(np.mean((predictions - y_test) ** 2))
            
            logger.info(f"Testing complete. RMSE: {rmse}")
            return rmse, predictions
        except Exception as e:
            logger.error(f"Error during model testing: {e}")
            raise
    
    
    
                
                
    def log_to_mlflow(self, rmse, predictions):        
        model = load_model(self.config.model_path)  

        try:
            mlflow.set_registry_uri(self.config.mlflow_uri)
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
            print("Tracking URI: ", mlflow.get_tracking_uri())
            with mlflow.start_run():
                # Log metrics
                mlflow.log_metric("RMSE", rmse)
                # Log parameters
                if tracking_url_type_store != "file":
                    # track in the remote server
                    print("tracking url: ",tracking_url_type_store)
                    print("***remote***\n")
                    # Log model
                    mlflow.keras.log_model(model, artifact_path="model")
                else:
                    # track in the local
                    print("***local***\n")
                    mlflow.keras.log_model(model, "model")
        except Exception as e:
            logger.error(f"Error logging to MLflow: {e}")
            raise

    def save_rmse(self, rmse: float):
        try:
            os.makedirs(os.path.dirname(self.config.rmse_path), exist_ok=True)
            with open(self.config.rmse_path, 'w') as f:
                f.write(str(rmse))
            logger.info(f"RMSE saved to {self.config.rmse_path}")
        except Exception as e:
            logger.error(f"Error saving RMSE: {e}")
            raise


In [8]:
config_manager = ConfigurationManager(config_filepath=CONFIG_FILE_PATH)
evaluation_config = config_manager.get_model_evaluation_config()
evaluator = ModelEvaluator(config=evaluation_config)


[2024-11-26 18:33:34,548: INFO: common: yaml file: config\config.yaml loaded successfully]


In [10]:
data = evaluator.fetch_real_time_data()

[2024-11-26 18:34:52,329: INFO: 1115350102: Real-time data fetched successfully.]


In [11]:
data.shape

(100, 5)

In [12]:
scaled_data, dataset = evaluator.fetch_and_preprocess_real_time_data()

[2024-11-26 18:35:05,373: INFO: 1115350102: Real-time data fetched successfully.]
[2024-11-26 18:35:05,373: INFO: 1115350102: Real-time data fetched for symbol: IBM]
[2024-11-26 18:35:05,389: INFO: 1115350102: Real-time data scaled.]


In [13]:
len(scaled_data)

100

In [14]:
# Test model on real-time data
logger.info("Testing model on real-time data...")
predictions = evaluator.test_model_real_time(scaled_data)

[2024-11-26 18:35:07,096: INFO: 1618695276: Testing model on real-time data...]
2/2 [==============================] - 2s 65ms/step
[2024-11-26 18:35:09,986: INFO: 1115350102: Real-time model testing complete.]


In [15]:
predictions.shape

(40, 1)

In [16]:
scaled_data.shape

(100, 1)

In [17]:
dataset.shape

(100, 1)

In [18]:
rmse, test_predictions = evaluator.test_model(scaled_data, len(scaled_data), dataset)

x_test shape: (40, 60, 1)
2/2 [==============================] - 1s 10ms/step
prediction mean: 226.0263671875
y_test mean: 225.818075
[2024-11-26 18:35:15,198: INFO: 1115350102: Testing complete. RMSE: 0.3591865693837794]


In [2]:
import os

In [4]:
os.environ.get('MLFLOW_TRACKING_URI')
os.environ.get("MLFLOW_TRACKING_USERNAME")
os.environ.get("MLFLOW_TRACKING_PASSWORD")

In [20]:
# Verify the variables (optional)
print("MLFLOW_TRACKING_URI:", os.environ.get("MLFLOW_TRACKING_URI"))
print("MLFLOW_TRACKING_USERNAME:", os.environ.get("MLFLOW_TRACKING_USERNAME"))
print("MLFLOW_TRACKING_PASSWORD:", os.environ.get("MLFLOW_TRACKING_PASSWORD"))

MLFLOW_TRACKING_URI: https://dagshub.com/BouazizOmar/Automated-Stock-Price-Prediction-Pipeline-A-Complete-MLOps-Lifecycle.mlflow
MLFLOW_TRACKING_USERNAME: BouazizOmar
MLFLOW_TRACKING_PASSWORD: a3c5b006173123d93ef056eb8b6f9fde0792be3f


In [ ]:
logger.info("Logging results to MLflow...")
try:
    model = load_model(evaluation_config.model_path)
    evaluator.log_to_mlflow(model=model, rmse=rmse, predictions=test_predictions)
    logger.info("Logged to MLflow successfully.")
except Exception as e:
    logger.error(f"Error logging to MLflow: {e}")

[2024-11-26 18:35:31,381: INFO: 3386568819: Logging results to MLflow...]
Tracking URI:  https://dagshub.com/BouazizOmar/Automated-Stock-Price-Prediction-Pipeline-A-Complete-MLOps-Lifecycle.mlflow


2024/11/26 18:35:33 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


tracking url:  https
***remote***

[2024-11-26 18:35:42,591: INFO: builder_impl: Assets written to: C:\Users\Portable\AppData\Local\Temp\tmpllu2k1_w\model\data\model\assets]


In [9]:
import streamlit as st
from src.StockPricePrediction.components.model_evaluation_mlflow import ModelEvaluator
from src.StockPricePrediction.config.configuration import ConfigurationManager
from src.StockPricePrediction.constants import CONFIG_FILE_PATH
from src.StockPricePrediction import logger

In [ ]:
# Initialize ConfigurationManager
config_manager = ConfigurationManager(config_filepath=CONFIG_FILE_PATH)
evaluation_config = config_manager.get_model_evaluation_config()

# Initialize Pipeline
evaluator = ModelEvaluator(config=evaluation_config)
data = evaluator.fetch_real_time_data()
scaled_data, dataset = evaluator.fetch_and_preprocess_real_time_data()

[2024-11-27 11:45:55,149: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-27 11:45:56,854: INFO: model_evaluation_mlflow: Real-time data fetched successfully.]
[2024-11-27 11:45:57,486: INFO: model_evaluation_mlflow: Real-time data fetched successfully.]
[2024-11-27 11:45:57,487: INFO: model_evaluation_mlflow: Real-time data fetched for symbol: IBM]
[2024-11-27 11:45:57,517: INFO: model_evaluation_mlflow: Real-time data scaled.]


In [13]:
# Streamlit App
st.title("Stock Price Prediction with LSTM")
st.markdown("""
This application predicts the next stock price using a trained LSTM model. It also logs metrics and models to MLflow.
""")
rmse, test_predictions = evaluator.test_model(scaled_data, len(scaled_data), dataset)

# Display current stock price
st.subheader("Current Stock Price")
try:
    latest_close_price = data["close"].iloc[-1]
    st.write(f"**Current Stock Price ({evaluation_config.symbol}):** ${latest_close_price:.2f}")
except Exception as e:
    logger.error(f"Error fetching current stock price: {e}")
    st.error("Could not fetch the current stock price. Check the logs for details.")

2024-11-27 11:47:01.089 
  command:

    streamlit run c:\Users\Portable\anaconda3\envs\MLOpsProject\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


x_test shape: (40, 60, 1)
2/2 [==============================] - 1s 0s/step
prediction mean: 228.95394897460938
y_test mean: 228.86881750000003
[2024-11-27 11:47:03,629: INFO: model_evaluation_mlflow: Testing complete. RMSE: 0.34327656665254136]
Tracking URI:  https://dagshub.com/BouazizOmar/Automated-Stock-Price-Prediction-Pipeline-A-Complete-MLOps-Lifecycle.mlflow


2024/11/27 11:47:06 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


tracking url:  https
***remote***

[2024-11-27 11:47:11,377: INFO: builder_impl: Assets written to: C:\Users\Portable\AppData\Local\Temp\tmp6lgk3r5h\model\data\model\assets]


2024/11/27 11:47:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/27 11:49:02 INFO mlflow.tracking._tracking_service.client: 🏃 View run placid-mink-441 at: https://dagshub.com/BouazizOmar/Automated-Stock-Price-Prediction-Pipeline-A-Complete-MLOps-Lifecycle.mlflow/#/experiments/0/runs/c52d534b197e4beab3586339130c7346.
2024/11/27 11:49:02 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/BouazizOmar/Automated-Stock-Price-Prediction-Pipeline-A-Complete-MLOps-Lifecycle.mlflow/#/experiments/0.


[2024-11-27 11:49:02,468: INFO: 1628663338: Logged to MLflow successfully.]


In [14]:
scaled_data, dataset = evaluator.fetch_and_preprocess_real_time_data()

[2024-11-27 11:55:06,488: INFO: model_evaluation_mlflow: Real-time data fetched successfully.]
[2024-11-27 11:55:06,495: INFO: model_evaluation_mlflow: Real-time data fetched for symbol: IBM]
[2024-11-27 11:55:06,529: INFO: model_evaluation_mlflow: Real-time data scaled.]


In [15]:
predictions = evaluator.test_model_real_time(scaled_data)

2/2 [==============================] - 1s 7ms/step
[2024-11-27 11:55:10,963: INFO: model_evaluation_mlflow: Real-time model testing complete.]


In [16]:
predictions

array([[229.2669 ],
       [229.27754],
       [229.27519],
       [229.26129],
       [229.24379],
       [229.22469],
       [229.20555],
       [229.1748 ],
       [229.09624],
       [229.0237 ],
       [228.9746 ],
       [228.89398],
       [228.84634],
       [228.8234 ],
       [228.83284],
       [228.84085],
       [228.83073],
       [228.80841],
       [228.81808],
       [228.83525],
       [228.85638],
       [228.879  ],
       [228.8928 ],
       [228.90788],
       [228.92287],
       [228.93721],
       [228.9383 ],
       [228.9379 ],
       [228.9264 ],
       [228.92313],
       [228.92561],
       [228.90689],
       [228.87488],
       [228.85167],
       [228.84369],
       [228.84671],
       [228.8326 ],
       [228.80748],
       [228.79488],
       [228.79744]], dtype=float32)

In [17]:
next_price = predictions[-1]  # Last prediction

In [19]:
next_price[0]

228.79744

In [ ]:
print(f"next_price: {next_price}")
st.write(f"**Predicted Stock Price for Next Period:** ${next_price:.2f}")


In [ ]:
# Predict next stock price
st.subheader("Predicted Next Stock Price")
try:
    scaled_data, dataset = evaluator.fetch_and_preprocess_real_time_data()
    predictions = evaluator.test_model_real_time(scaled_data)
    next_price = predictions[-1]  # Last prediction
    print(f"next_price: {next_price}")
    st.write(f"**Predicted Stock Price for Next Period:** ${next_price:.2f}")
except Exception as e:
    logger.error(f"Error predicting next stock price: {e}")
    st.error("Could not predict the next stock price. Check the logs for details.")

# Model Evaluation and Logging
st.subheader("Model Evaluation Metrics")
try:
    # Perform model evaluation
    rmse, test_predictions = evaluator.test_model(scaled_data, len(dataset), dataset)

    # Display RMSE
    st.write(f"**Root Mean Square Error (RMSE):** {rmse:.2f}")

    # Log metrics and model to MLflow
    evaluator.log_to_mlflow(rmse=rmse, predictions=test_predictions)
    st.success("Model metrics and artifact logged to MLflow successfully!")
except Exception as e:
    logger.error(f"Error during model evaluation and logging: {e}")
    st.error("Could not evaluate the model or log to MLflow. Check the logs for details.")
